In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,59,2024-03-01,-1,107.655172,97.389655,0.526448,11.915517,21.822414,0.168414,110.212069,...,10.055172,24.650000,0.239207,119.081034,0,3,4,3,5,2024
1,59,2024-03-01,-1,114.706897,97.231034,0.559862,11.594828,24.136207,0.185483,117.236207,...,13.144828,25.005172,0.193862,112.546552,0,3,4,2,6,2024
2,60,2024-03-01,1,118.898305,99.828814,0.564797,10.655932,22.076271,0.198831,119.093220,...,10.631034,24.465517,0.199603,123.248276,0,3,4,2,7,2024
3,58,2024-03-01,1,119.245614,99.082456,0.556982,12.084211,27.487719,0.189509,119.103509,...,11.310169,25.177966,0.186356,115.544068,0,3,4,3,10,2024
4,61,2024-03-01,1,123.916667,101.691667,0.583917,11.176667,23.503333,0.177267,121.861667,...,11.406667,25.528333,0.209717,118.448333,0,3,4,2,12,2024
5,59,2024-03-01,-2,118.103448,99.137931,0.564138,11.298276,23.491379,0.173086,118.008621,...,12.796610,22.994915,0.219898,116.777966,0,3,4,2,26,2024
6,58,2024-03-01,-9,107.245614,96.705263,0.502772,12.436842,26.524561,0.195789,109.791228,...,12.518644,21.952542,0.181915,108.442373,0,3,4,3,25,2024
7,60,2024-03-01,3,121.813559,100.508475,0.575102,11.072881,21.910169,0.220593,120.561017,...,10.574576,24.815254,0.188712,115.266102,0,3,4,3,17,2024
8,59,2024-03-01,-12,114.327586,102.750000,0.538103,11.837931,19.950000,0.173793,111.256897,...,11.285965,24.808772,0.216895,120.791228,0,3,4,3,30,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0
1,2024-03-01,CLE,DET,1.253165,4.100000,1
2,2024-03-01,DAL,BOS,4.100000,1.253165,0
3,2024-03-01,GSW,TOR,1.645161,2.300000,0
4,2024-03-01,IND,NOP,2.700000,1.487805,0
5,2024-03-01,SAC,MIN,3.000000,1.408163,0
6,2024-03-01,POR,MEM,1.952381,1.869565,0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1
8,2024-03-01,WAS,LAC,9.500000,1.068966,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0
1,2024-03-01,CLE,DET,1.253165,4.100000,1
2,2024-03-01,DAL,BOS,4.100000,1.253165,0
3,2024-03-01,GSW,TOR,1.645161,2.300000,0
4,2024-03-01,IND,NOP,2.700000,1.487805,0
5,2024-03-01,SAC,MIN,3.000000,1.408163,0
6,2024-03-01,POR,MEM,1.952381,1.869565,0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1
8,2024-03-01,WAS,LAC,9.500000,1.068966,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0
1,2024-03-01,CLE,DET,1.253165,4.100000,1
2,2024-03-01,DAL,BOS,4.100000,1.253165,0
3,2024-03-01,GSW,TOR,1.645161,2.300000,0
4,2024-03-01,IND,NOP,2.700000,1.487805,0
5,2024-03-01,SAC,MIN,3.000000,1.408163,0
6,2024-03-01,POR,MEM,1.952381,1.869565,0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1
8,2024-03-01,WAS,LAC,9.500000,1.068966,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0
1,2024-03-01,CLE,DET,1.253165,4.100000,1
2,2024-03-01,DAL,BOS,4.100000,1.253165,0
3,2024-03-01,GSW,TOR,1.645161,2.300000,1
4,2024-03-01,IND,NOP,2.700000,1.487805,0
5,2024-03-01,SAC,MIN,3.000000,1.408163,0
6,2024-03-01,POR,MEM,1.952381,1.869565,0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1
8,2024-03-01,WAS,LAC,9.500000,1.068966,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0.0
1,2024-03-01,CLE,DET,1.253165,4.100000,1.0
2,2024-03-01,DAL,BOS,4.100000,1.253165,0.0
3,2024-03-01,GSW,TOR,1.645161,2.300000,1.0
4,2024-03-01,IND,NOP,2.700000,1.487805,1.0
5,2024-03-01,SAC,MIN,3.000000,1.408163,1.0
6,2024-03-01,POR,MEM,1.952381,1.869565,0.0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1.0
8,2024-03-01,WAS,LAC,9.500000,1.068966,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-01,CHO,PHI,5.400000,1.166667,0.0
1,2024-03-01,CLE,DET,1.253165,4.100000,1.0
2,2024-03-01,DAL,BOS,4.100000,1.253165,0.0
3,2024-03-01,GSW,TOR,1.645161,2.300000,1.0
4,2024-03-01,IND,NOP,2.700000,1.487805,1.0
5,2024-03-01,SAC,MIN,3.000000,1.408163,1.0
6,2024-03-01,POR,MEM,1.952381,1.869565,0.0
7,2024-03-01,MIL,CHI,1.555556,2.500000,1.0
8,2024-03-01,WAS,LAC,9.500000,1.068966,0.0
